# Creating FITS output for any given SME output

## Author(s): Sven Buder (SB, WG4)

### History:
181011 SB Created

In [1]:
# Preamble for notebook 

# Compatibility with Python 3
from __future__ import (absolute_import, division, print_function)

try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

# Basic packages
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import os
import sys
import scipy
import scipy.interpolate
import collections
import glob
import pickle
import pandas
from astropy.stats import sigma_clipped_stats

# Packages to work with FITS and (IDL) SME.out files
import astropy.io.fits as pyfits
import astropy.table as table
from astropy.table import Table, hstack, vstack
from scipy.io.idl import readsav
from astropy.nddata import bitmask

# Matplotlib and associated packages for plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.transforms import Bbox,TransformedBbox
from matplotlib.image import BboxImage
from matplotlib.legend_handler import HandlerBase
from matplotlib._png import read_png
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors

params = {
    'font.family'        : 'sans',
    'font.size'          : 17,
    'axes.labelsize'     : 20,
    'ytick.labelsize'    : 16,
    'xtick.labelsize'    : 16,
    'legend.fontsize'    : 20,
    'text.usetex'        : True, 
    'text.latex.preamble': [r'\usepackage{upgreek}', r'\usepackage{amsmath}'],
    }   
plt.rcParams.update(params)

_parula_data = [[0.2081, 0.1663, 0.5292], 
                [0.2116238095, 0.1897809524, 0.5776761905], 
                [0.212252381, 0.2137714286, 0.6269714286], 
                [0.2081, 0.2386, 0.6770857143], 
                [0.1959047619, 0.2644571429, 0.7279], 
                [0.1707285714, 0.2919380952, 0.779247619], 
                [0.1252714286, 0.3242428571, 0.8302714286], 
                [0.0591333333, 0.3598333333, 0.8683333333], 
                [0.0116952381, 0.3875095238, 0.8819571429], 
                [0.0059571429, 0.4086142857, 0.8828428571], 
                [0.0165142857, 0.4266, 0.8786333333], 
                [0.032852381, 0.4430428571, 0.8719571429], 
                [0.0498142857, 0.4585714286, 0.8640571429], 
                [0.0629333333, 0.4736904762, 0.8554380952], 
                [0.0722666667, 0.4886666667, 0.8467], 
                [0.0779428571, 0.5039857143, 0.8383714286], 
                [0.079347619, 0.5200238095, 0.8311809524], 
                [0.0749428571, 0.5375428571, 0.8262714286], 
                [0.0640571429, 0.5569857143, 0.8239571429], 
                [0.0487714286, 0.5772238095, 0.8228285714], 
                [0.0343428571, 0.5965809524, 0.819852381], 
                [0.0265, 0.6137, 0.8135], 
                [0.0238904762, 0.6286619048, 0.8037619048], 
                [0.0230904762, 0.6417857143, 0.7912666667], 
                [0.0227714286, 0.6534857143, 0.7767571429], 
                [0.0266619048, 0.6641952381, 0.7607190476], 
                [0.0383714286, 0.6742714286, 0.743552381], 
                [0.0589714286, 0.6837571429, 0.7253857143], 
                [0.0843, 0.6928333333, 0.7061666667], 
                [0.1132952381, 0.7015, 0.6858571429], 
                [0.1452714286, 0.7097571429, 0.6646285714], 
                [0.1801333333, 0.7176571429, 0.6424333333], 
                [0.2178285714, 0.7250428571, 0.6192619048], 
                [0.2586428571, 0.7317142857, 0.5954285714], 
                [0.3021714286, 0.7376047619, 0.5711857143], 
                [0.3481666667, 0.7424333333, 0.5472666667], 
                [0.3952571429, 0.7459, 0.5244428571], 
                [0.4420095238, 0.7480809524, 0.5033142857], 
                [0.4871238095, 0.7490619048, 0.4839761905], 
                [0.5300285714, 0.7491142857, 0.4661142857], 
                [0.5708571429, 0.7485190476, 0.4493904762],
                [0.609852381, 0.7473142857, 0.4336857143], 
                [0.6473, 0.7456, 0.4188], 
                [0.6834190476, 0.7434761905, 0.4044333333], 
                [0.7184095238, 0.7411333333, 0.3904761905], 
                [0.7524857143, 0.7384, 0.3768142857], 
                [0.7858428571, 0.7355666667, 0.3632714286], 
                [0.8185047619, 0.7327333333, 0.3497904762], 
                [0.8506571429, 0.7299, 0.3360285714], 
                [0.8824333333, 0.7274333333, 0.3217], 
                [0.9139333333, 0.7257857143, 0.3062761905], 
                [0.9449571429, 0.7261142857, 0.2886428571], 
                [0.9738952381, 0.7313952381, 0.266647619], 
                [0.9937714286, 0.7454571429, 0.240347619], 
                [0.9990428571, 0.7653142857, 0.2164142857], 
                [0.9955333333, 0.7860571429, 0.196652381], 
                [0.988, 0.8066, 0.1793666667], 
                [0.9788571429, 0.8271428571, 0.1633142857], 
                [0.9697, 0.8481380952, 0.147452381], 
                [0.9625857143, 0.8705142857, 0.1309], 
                [0.9588714286, 0.8949, 0.1132428571], 
                [0.9598238095, 0.9218333333, 0.0948380952], 
                [0.9661, 0.9514428571, 0.0755333333], 
                [0.9763, 0.9831, 0.0538]]

parula = ListedColormap(_parula_data, name='parula')
parula_zero = _parula_data[0]
parula_0 = ListedColormap(_parula_data, name='parula_0')
parula_0.set_bad((1,1,1))
parula_r = ListedColormap(_parula_data[::-1], name='parula_r')

willi_blau = [0.0722666667, 0.4886666667, 0.8467]

class read_iso():

    def __init__(self):
        self.num_cols=4
        self.columns = ['M_Mo', 'logTeff', 'logG', 'logL_Lo', 'Ks']
        self.num_ages = len(age)
        self.ages = age

    def fill_chemistry(self, m_h, fe_h, alpha_fe):
        self.FeH = fe_h
        self.Z = 10**m_h*0.0152
        self.aFe = alpha_fe

    def fill_iso(self, iso_input):
        self.data = iso_input



## Function to gather and combine input data from SME, IRAF, Output structure & A(X)_sun

In [2]:
def get_product_information(product_name): 

    if product_name == 'GBS':
        product_subsets = 'gbs'
        product_input_data_path = '../validation/stellar_parameters/gaia_fgk_benchmark_stars/data/'
        product_pipeline = 'lbol'

    if product_name == 'seis':
        product_subsets = 'seis'
        product_input_data_path = '../input/asteroseismic_information/'
        product_pipeline = 'lbol'

    if product_name == 'OpenClusters':
        product_subsets = ['Blanco 1','Pleiades','Hyades','NGC 1817','NGC 1901','ASCC 16','ASCC 20','ASCC 21','NGC 2112','NGC 2204','Berkeley 73','NGC 2232','NGC 2243','Berkeley 33','Berkeley 32','NGC 2516','NGC 2548','NGC 2632','M 67','IC 2602','Melotte 101','Trumpler 20','NGC 5460','NGC 6253','ASCC 89','IC 4665','NGC 6469','NGC 6568','NGC 6583','Ruprecht 145','Ruprecht 147']
        product_input_data_path = '../validation/comparisons/comparison_clusters/sme_output_files/'
        product_pipeline = 'lbol'

    if product_name == 'GlobularClusters':
        product_subsets = ['47 Tuc','NGC 288','NGC 362','NGC 1851','Omega Cen','NGC 6362','NGC 6397','NGC 7099']
        product_input_data_path = '../validation/comparisons/comparison_clusters/sme_output_files/'
        product_pipeline = 'lbol'

    if product_name == 'random10000':
        product_subsets = '10000'
        product_input_data_path = '../validation/random10000/data/'
        product_pipeline = 'lbol'

    if product_name == 'ts_DR2':
        product_subsets = 'ts_DR2'
        product_input_data_path = '../validation/DR2_rerun/data/'
        product_pipeline = 'lbol'

    if product_name == 'high_vtot':
        product_subsets = 'high_vtot'
        product_input_data_path = '../science/high_vtot/data/'
        product_pipeline = 'lbol'

    if product_name == 'Li_rich_giants':
        product_subsets = 'Li_rich'
        product_input_data_path = '../science/Li_rich_giants/data/'
        product_pipeline = 'lbol'

    if product_name == 'Keller':
        product_subsets = 'Keller'
        product_input_data_path = '../science/Keller/'
        product_pipeline = 'lbol'


    if product_name == 'wide_binaries':
        product_subsets = 'wide_binaries'
        product_input_data_path = '../science/wide_binaries/data/'
        product_pipeline = 'lbol'

    if product_name[:4] == '10k_':
        product_subsets = product_name
        product_input_data_path = 'sme_result_files/'
        product_pipeline = 'lbol_final'

    if (
        (product_name[:2] == '13') | 
        (product_name[:2] == '14') | 
        (product_name[:2] == '15') | 
        (product_name[:2] == '16') | 
        (product_name[:2] == '17') | 
        (product_name[:2] == '18') |
        (product_name[:2] == '19') |
        (product_name[:2] == '20') 
        ):
        product_subsets = product_name
        product_input_data_path = 'sme_result_files/'
        product_pipeline = 'lbol'

    return (product_subsets, product_input_data_path, product_pipeline)

In [3]:
def get_input_data(product_subsets, product_input_data_path, product_pipeline): 
    
    dr3_output_structure = Table.read('galah_dr3_output_structure.fits',1)

    if np.shape(product_subsets)!=():
        joined_sme_data = Table.read(product_input_data_path+'/GALAH_'+product_subsets[0].replace(" ", "")+'_'+product_pipeline+'.fits', format='fits')
        for each in range(1,len(product_subsets)):
            joined_sme_data = vstack([joined_sme_data, Table.read(product_input_data_path+'/GALAH_'+product_subsets[each].replace(" ", "")+'_'+product_pipeline+'.fits', format='fits')])
    else:
        joined_sme_data = Table.read(product_input_data_path+'/GALAH_'+product_subsets.replace(" ", "")+'_'+product_pipeline+'.fits', format='fits')
        
    all_iraf_data = pyfits.getdata('../input/sobject_iraf_53_2MASS_GaiaDR2_WISE_PanSTARRSDR1_BailerJones_K2seis_small.fits',1)
    iraf_matched = []
    for each in joined_sme_data['SOBJECT_ID']:
        match = np.where(each == all_iraf_data['sobject_id'])[0]
        if len(match)>0:
            iraf_matched.append(match[0])
        else:
            print('No match found for '+str(each))
    iraf_matched = np.array(iraf_matched)
    iraf_data = all_iraf_data[iraf_matched]
    
    abundance_zeropoints= Table.read('../validation/abundances/galahdr3_abundance_zeropoints.fits',1)
    
    print('Got input data')
    
    return dr3_output_structure, joined_sme_data, iraf_data, abundance_zeropoints

In [4]:
def compute_final_uncertainty(final_output_data, each_key): 
    """
    Combine the three uncertainty types for parameter X:
    
    e_final^2 (X) = e_fit^2(X) + e_precision^2(X) + e_accuracy^2(X)
    """
    
    uncertainty_accuracy = collections.OrderedDict()
    uncertainty_accuracy['teff'] = 67
    uncertainty_accuracy['logg'] = 0.12
    uncertainty_accuracy['fe_h'] = 0.034
    uncertainty_accuracy['fe_h_atmo'] = 0.059
    uncertainty_accuracy['vbroad'] = 2.0    
    uncertainty_accuracy['rv_galah'] = 0.1
    # This value is fixed now
    # for more details see the validation/radial_velocities part

    print('NB: We are leaving out logg for now, because we compute its final uncertainty later')

    opt_parms = np.load('../validation/repeat_observations/reference_uncertainties/repeats_'+each_key+'.npy')

    def snr_sigma_func(t, A, K, C):
        return A * np.exp(- K * t) + C

    snr_in = final_output_data['snr_c2_iraf'].clip(min=0)

    scaling = dict()
    scaling['teff'] = [3,7.5]
    scaling['logg'] = [2,0]
    scaling['fe_h'] = [4.,0.01]
    scaling['fe_h_atmo'] = [2,0.0125]
    scaling['vbroad'] = [1.75,0.3]
    scaling['rv_galah'] = [2.0,0.15]
    
    new_error = np.sqrt(
        # SME fit uncertainty
        (scaling[each_key][0] * final_output_data['cov_e_'+each_key] + scaling[each_key][1])**2.+
        # Precision uncertainty, interpolated for the CCD2 SNR
        (
        snr_sigma_func(snr_in,*opt_parms)
        )**2.+
        # Accuracy uncertainty
        (uncertainty_accuracy[each_key])**2.
    )

    new_error[new_error==0] = np.nan

    final_output_data['e_'+each_key] = new_error

    return(final_output_data)

In [5]:
def compute_final_abundance_uncertainty(final_output_data, final_output_abundances):
    for each_key in final_output_abundances.keys():
        if (each_key[:2] == 'A_') & (each_key[2:4] != 'Fe') & (each_key[2:] not in ['Ti6599','Cr4848']):
            
            try:
                opt_parms = np.load('../validation/repeat_observations/reference_uncertainties/repeats_'+each_key[2:]+'.npy')
            except:
                print('No reference repeat uncertainty for '+str(each_key[2:]))
                opt_parms = np.array([0.,0.,0.])
                
            def snr_sigma_func(t, A, K, C):
                return A * np.exp(- K * t) + C

            snr_in = final_output_data['snr_c2_iraf'].clip(min=0)
            
            
            final_output_data['cov_e_'+each_key[2:]] = np.nanmax([
                # SME covariance uncertainty
                final_output_data['cov_e_'+each_key[2:]],
                # Precision uncertainty, interpolated for the CCD2 SNR
                (
                snr_sigma_func(snr_in,*opt_parms)
                )
            ], axis=0
            )
    
    return(final_output_data, final_output_abundances)

In [6]:
def compute_logg_uncertainty(final_output_data, mc_sampling=10000): 
    
    print('Sampling logg uncertainty')
    
    np.random.seed(12)
    
    mc_teff = np.random.normal(
        loc = final_output_data['teff'],
        # We do not trust the SME e_teff, and hence sample the max of e_teff=100 and 2% error of teff
        # scale = np.array([np.max([100,0.02*final_output_data['teff'][x]]) for x in range(len(final_output_data['teff']))]),
        scale = final_output_data['e_teff'],
        size=(mc_sampling,len(final_output_data['teff']))
    )
    mc_mass = np.random.normal(
        loc = final_output_data['mass'],
        # We do not know the error on mass and hence assume it is 10%
        scale = 0.1*final_output_data['mass'],
        size=(mc_sampling,len(final_output_data['mass']))
    )
    mc_mass.clip(min=0., out=mc_mass)
    mc_kmag = np.random.normal(
        loc = final_output_data['ks_m'],
        scale = final_output_data['ks_msigcom'],
        size=(mc_sampling,len(final_output_data['ks_m']))
    )
    mc_bc = np.random.normal(
        loc = final_output_data['bc_ks'],
        scale = 0.1*np.ones(len(final_output_data['bc_ks'])),
        size=(mc_sampling,len(final_output_data['bc_ks']))
    )
    
    sigma_dist_hi = abs(final_output_data['r_hi'] - final_output_data['r_est'])
    sigma_dist_lo = abs(final_output_data['r_est'] - final_output_data['r_lo'])
    mc_dist_lo = np.abs(np.random.normal(
        loc = 0,
        scale = sigma_dist_lo,
        size=(mc_sampling,len(final_output_data['r_est'])))
    )
    mc_dist_hi = np.abs(np.random.normal(
        loc = 0,
        scale = sigma_dist_hi,
        size=(mc_sampling,len(final_output_data['r_est'])))
    )
    #fraction_hi_lo = sigma_dist_hi / (sigma_dist_lo + sigma_dist_hi)
    fraction_hi_lo = 0.5
    select_dist_lo_hi = (np.random.uniform(0, 1, size=(mc_sampling,len(final_output_data['r_est']))) < fraction_hi_lo)
    mc_dist = np.array(final_output_data['r_est']).T + select_dist_lo_hi*mc_dist_hi - (1-select_dist_lo_hi)*mc_dist_lo
    mc_dist.clip(min=0.001, out=mc_dist)
    e_a_ks = (final_output_data['e_a_ks']).clip(min=0.01)
    mc_ak = np.random.normal(
        loc = final_output_data['a_ks'],
        scale = e_a_ks,
        size=(mc_sampling,len(final_output_data['a_ks']))
    )
    mc_ak.clip(min=0., out=mc_ak)
    
    def logg_function(teff, mass, kmag, bc, dist, ak):
        return(4.438 + 4*np.log10(teff/5772.) + np.log10(mass) + 0.4*(kmag + bc - 5.*np.log10(dist) + 5 - ak - 4.7554))

    mc_logg = logg_function(mc_teff, mc_mass, mc_kmag, mc_bc, mc_dist, mc_ak)

    logg_mean = np.array([np.nanmean(mc_logg[:,x]) for x in range(np.shape(mc_logg)[1])])
    logg_std  = np.array([np.nanstd(mc_logg[:,x]) for x in range(np.shape(mc_logg)[1])])

    print('Done sampling logg uncertainty')

    return(logg_std)

In [7]:
def combine_SME_IRAF_to_FINAL(output_filename, product_pipeline, sme_data, iraf_data, dr3_output_structure, abundance_zeropoints):
    
    print('Combining information')
    
    final_output_data = collections.OrderedDict()
    final_output_abundances = collections.OrderedDict()
    
    abundances_in_mode = np.array([(sme_data['MODE'][0,it]).replace(" ","") for it in range(len(sme_data['MODE'][0]))])
    
    for each_key in dr3_output_structure.keys():
        
        # Keys in output not matching input keys
        if each_key=='star_id':
            final_output_data[each_key]=np.array(iraf_data['tmass_id'])
            final_output_abundances[each_key]=np.array(iraf_data['tmass_id'])
        elif each_key in ['sobject_id']:
            final_output_data[each_key]=np.array(sme_data[each_key.upper()])
            final_output_abundances[each_key]=np.array(sme_data[each_key.upper()])
        elif each_key in ['field_id','source_id','ra','ra_error','dec','dec_error','l','b','r_est','r_lo','r_hi','pmra','pmra_error','pmdec','pmdec_error','ra_dec_corr','ra_parallax_corr','ra_pmra_corr','ra_pmdec_corr','dec_parallax_corr','dec_pmra_corr','dec_pmdec_corr','parallax_pmra_corr','parallax_pmdec_corr','pmra_pmdec_corr','red_flag','ebv','snr_c1_iraf','snr_c2_iraf','snr_c3_iraf','snr_c4_iraf','flag_guess','rv_guess','e_rv_guess','teff_guess','logg_guess','feh_guess','j_m','j_msigcom','h_m','h_msigcom','ks_m','ks_msigcom','ph_qual_tmass','w2mpro','w2mpro_error','ph_qual_wise','parallax','parallax_error','visibility_periods_used','astrometric_chi2_al','astrometric_n_good_obs_al','ruwe','phot_g_mean_mag','bp_rp']:
            final_output_data[each_key]=np.array(iraf_data[each_key])
        elif each_key=='wg4_field':
            if product_pipeline in ['lbol','lbol_final']:
                final_output_data[each_key]=np.array([sme_data['FIELD'][x][:-5] for x in range(len(sme_data['FIELD']))])
                final_output_data['wg4_pipeline']=np.array(['lbol' for x in range(len(sme_data['FIELD']))])
            elif product_pipeline == 'seis':
                final_output_data[each_key]=np.array([sme_data['FIELD'][x][:-5] for x in range(len(sme_data['FIELD']))])
                final_output_data['wg4_pipeline']=np.array(['seis' for x in range(len(sme_data['FIELD']))])
            else:
                final_output_data[each_key]=np.array(sme_data['FIELD'])
                final_output_data['wg4_pipeline']=np.array(['free' for x in range(len(sme_data['FIELD']))])
        elif each_key in ['flag_sp','teff', 'e_teff', 'logg', 'e_logg', 'vmic', 'e_vmic', 'mass', 'lbol', 'age']:
            final_output_data[each_key]=np.array(sme_data[each_key.upper()])
        elif each_key[:-3] in ['alpha','LiI','CI','OI','NaI','MgI',
                              'AlI','SiI','KI','CaI','ScI','ScII',
                              'TiI','TiII','VI','CrI','CrII','MnI',
                              #'FeI','FeII',
                              'CoI','NiI','CuI','ZnI',
                              'RbI','SrI','YII','ZrI','MoI','RuI',
                              'BaII','LaII','CeII','NdII','SmII','EuII']:
            final_output_data[each_key] = np.array([np.NaN for x in range(len(sme_data['FIELD']))])
            final_output_data['e_'+each_key] = np.array([np.NaN for x in range(len(sme_data['FIELD']))])
            final_output_data['nr_'+each_key] = np.zeros(len(sme_data['FIELD']))
            final_output_data['flag_'+each_key] = np.zeros(len(sme_data['FIELD']))
        elif each_key == 'rv_5854':
            final_output_data[each_key]=np.array(sme_data['BA_VRAD'])
        elif each_key == 'rv_6708':
            final_output_data[each_key]=np.array(sme_data['LI_VRAD'])
        elif each_key == 'rv_6722':
            final_output_data[each_key]=np.array(sme_data['SI_VRAD'])
        elif each_key == 'bc_ks':
            final_output_data[each_key]=np.array(sme_data['BC_K'])
        elif each_key == 'fe_h_atmo':
            final_output_data[each_key]=np.array(sme_data['FEH'])
            final_output_abundances[each_key]=np.array(sme_data['FEH'])
        elif each_key == 'e_fe_h_atmo':
            final_output_data[each_key]=np.array(sme_data['E_FEH'])
            final_output_abundances[each_key]=np.array(sme_data['E_FEH'])
        elif each_key == 'cov_e_fe_h_atmo':
            final_output_data[each_key]=np.array(sme_data['C_FEH'])
            final_output_abundances[each_key]=np.array(sme_data['C_FEH'])
        elif each_key[:6]=='cov_e_':
            if each_key[6:] in ['teff', 'logg']:
                final_output_data[each_key]=np.array(sme_data['C_'+each_key[6:].upper()])
        elif each_key in ['init_teff','init_logg']:
            final_output_data[each_key]=np.array(sme_data['S'+each_key[5:].upper()][:,0])
        elif each_key == 'init_fe_h_atmo':
            final_output_data[each_key]=np.array(sme_data['SFEH'][:,0])
        elif each_key == 'init_vbroad':
            final_output_data[each_key]=np.array(sme_data['SVSINI'][:,0])
        elif each_key=='fe_h':
            final_output_data['fe_h']=np.array(sme_data['A_ABUND'][:,1] - abundance_zeropoints['A_Fe'][0])
            final_output_data['e_fe_h']=np.array(sme_data['E_ABUND'][:,1])                
            final_output_data['cov_e_fe_h']=np.array(sme_data['C_ABUND'][:,1])                
            final_output_data['flag_fe_h']=np.array(sme_data['AFLAG'][:,1])                
            final_output_abundances['fe_h']=np.array(sme_data['A_ABUND'][:,1] - abundance_zeropoints['A_Fe'][0])
            final_output_abundances['e_fe_h']=np.array(sme_data['E_ABUND'][:,1])                
            final_output_abundances['cov_e_fe_h']=np.array(sme_data['C_ABUND'][:,1])                
            final_output_abundances['flag_fe_h']=np.array(sme_data['AFLAG'][:,1])                
        elif each_key=='vbroad':
            final_output_data[each_key]=np.array(sme_data['VSINI'])
            final_output_data['e_'+each_key]=np.array(sme_data['E_VSINI'])
            final_output_data['cov_e_'+each_key]=np.array(sme_data['C_VSINI'])
        elif each_key=='rv_galah':
            final_output_data[each_key]=np.array(sme_data['VEL'])
            final_output_data['e_'+each_key]=np.array(sme_data['E_VEL'])
            final_output_data['cov_e_'+each_key]=np.array(sme_data['C_VEL'])
        elif each_key=='rv_gaia':
            final_output_data[each_key]=np.array(iraf_data['radial_velocity'])
            final_output_data['e_'+each_key]=np.array(iraf_data['radial_velocity_error'])
        elif each_key=='chi2_sp':
            final_output_data[each_key]=np.array(sme_data['CHI'][:,0])
        elif each_key=='a_ks':

            # Apply RJCE method
            rjce_ak = np.array(0.918*(iraf_data['h_m'] - iraf_data['w2mpro'] - 0.08))
            e_rjce_ak = 0.918*np.sqrt(iraf_data['h_msigcom']**2. + iraf_data['w2mpro_error']**2.)
            
            rjce_ak.clip(min=0.0, out=rjce_ak)
            
            # check if 2MASS Hmag and WISE W2mag have good quality
            tmass_adjusted = np.array(iraf_data['ph_qual_tmass'])
            tmass_adjusted[np.where(tmass_adjusted=='   ')[0]] = 'UUU'
            tmass_adjusted[np.where(tmass_adjusted=='')[0]] = 'UUU'
            wise_adjusted = np.array(iraf_data['ph_qual_wise'])
            wise_adjusted[np.where(wise_adjusted=='    ')[0]] = 'UUUU'
            wise_adjusted[np.where(wise_adjusted=='')[0]] = 'UUUU'
            h_m_qual = np.array([tmass_adjusted[x][1] == 'A' for x in range(len(tmass_adjusted))])
            w2_qual = np.array([wise_adjusted[x][1] == 'A' for x in range(len(wise_adjusted))])

            # if photometry bad, exchange a_k by ebv approximation
            bad_rjce = np.isnan(iraf_data['h_m']) | np.isnan(iraf_data['w2mpro']) | (h_m_qual==False) | (w2_qual==False)
            ebv_ak = 0.36*iraf_data['ebv']
            rjce_ak[bad_rjce] = ebv_ak[bad_rjce]
            e_rjce_ak[bad_rjce] = ebv_ak[bad_rjce]
            
            final_output_data['a_ks']=rjce_ak
            final_output_data['e_a_ks']=e_rjce_ak
            
            # adjust too large E(B-V) if photometry good
            bad_ebv = np.where(0.36*iraf_data['ebv'] > 3*rjce_ak)[0]
            if len(bad_ebv) > 0:
                final_output_data['ebv'][bad_ebv] = 2.78*rjce_ak[bad_ebv]
            
            # nearby stars with ebv=0 and a_ks=0
            nearby = np.where(iraf_data['r_est'] <= 100.)[0]
            final_output_data['ebv'][nearby] = 0.
            final_output_data['a_ks'][nearby] = 0.
            final_output_data['e_a_ks'][nearby] = 0.

        # Abundance keys
        elif (each_key[0:2]=='A_') & (each_key != 'A_Ks'):
            element = each_key[2:]
            element_in_sme_data = np.where(element == abundances_in_mode)[0]
            if np.shape(element_in_sme_data)[0] != 1:
                # print('Exchange values for '+element)
                if element == 'K': 
                    element_in_sme_data = np.where('K7699' == abundances_in_mode)[0]
                if element == 'Ca': 
                    element_in_sme_data = np.where('Ca5862' == abundances_in_mode)[0]
                if element == 'Cu': 
                    element_in_sme_data = np.where('Cu5782' == abundances_in_mode)[0]
                if element == 'Ba': 
                    element_in_sme_data = np.where('Ba5854' == abundances_in_mode)[0]
                    
            # only if the element exists in the SME data
            elif np.shape(element_in_sme_data)[0] == 1:
                element_in_sme_data=element_in_sme_data[0]
                final_output_abundances[each_key]=np.array(sme_data['A_ABUND'][:,element_in_sme_data])

                if element=='Li':
                    final_output_data[each_key]=np.array(sme_data['A_ABUND'][:,element_in_sme_data])

                if element in ['Fe', 'Li']:
                    final_output_data['flux_'+each_key]=np.array(sme_data['LINEFLUX'][:,element_in_sme_data])
                    final_output_data['chi_'+each_key]=np.array(sme_data['CHI'][:,element_in_sme_data])

                if element!='Fe':
                    try:
                        final_output_data[element+'_fe']=np.array(
                            # [X/Fe] = [X/H] - [Fe/H] = A(X) - A(X)_sun - (A(Fe) - A(Fe)_sun)
                            sme_data['A_ABUND'][:,element_in_sme_data]- abundance_zeropoints[each_key][0] 
                            - (final_output_abundances['A_Fe'] - abundance_zeropoints['A_Fe'][0])
                            )
                    except:
                        print('You are using A(Fe) = 7.45 + fe_h_atmo!')
                        final_output_data[element+'_fe']=np.array(
                            # [X/Fe] = [X/H] - [Fe/H] = A(X) - A(X)_sun - (A(Fe) - A(Fe)_sun)
                            sme_data['A_ABUND'][:,element_in_sme_data]- abundance_zeropoints[each_key][0] 
                            - (7.45 + final_output_data['fe_h_atmo'] - abundance_zeropoints['A_Fe'][0])
                            )
                    #final_output_data[element+'_fe']=np.array(sme_data['ABUND'][:,element_in_sme_data])
                    #final_output_data['e_'+element+'_fe']=np.array(sme_data['E_ABUND'][:,element_in_sme_data])
                    #final_output_data['e_'+element]=np.array(sme_data['E_ABUND'][:,element_in_sme_data])
                    final_output_data['cov_e_'+element]=np.array(sme_data['C_ABUND'][:,element_in_sme_data])
                    final_output_data['flag_'+element]=np.array(sme_data['AFLAG'][:,element_in_sme_data])
                final_output_abundances['flux_'+each_key]=np.array(sme_data['LINEFLUX'][:,element_in_sme_data])
                final_output_abundances['chi_'+each_key]=np.array(sme_data['CHI'][:,element_in_sme_data])

            else:
                print('Could not find element '+element+' in SME data')
                
        # Already filled
        elif ((each_key in ['wg4_pipeline', 'e_fe_h', 'e_rv_galah', 'c_rv_galah', 'e_rv_gaia', 'e_vbroad', 'e_a_ks']) | (each_key[:4]=='e_A_') | (each_key[:7]=='flag_A_') | (each_key[:7]=='flux_A_') | (each_key[:7]=='chi2_A_') | (each_key[-3:]=='_fe')):
            pass
        # Placeholder
        elif each_key in ['e_mass', 'e_lbol', 'e_age', 'e_bc_ks']:
            final_output_data[each_key] = np.array([np.NaN for x in range(len(sme_data['FIELD']))])
        else:
            print('No match for '+each_key)

    return(final_output_data, final_output_abundances)

In [8]:
def combine_line_by_line(
    final_output_data, 
    final_output_abundances, 
    abundance_zeropoints,
    abundance_uncertainty_limit = 0.005,
    clip_outlier_sigma = 2,
    debug = False
    ):
    
    """
    We combine all line measurements 
    for a given element and species 
    as outlined in the converter dictionary, 
    e.g. OI is a combination of O7772, O7774, O7775
        
    Abundance uncertainties are set be 
    at least above the abundance_uncertainty_limit
    
    Only use those measurements that are as close as 2 sigma
    to the error-weighted mean and recompute it

    
    """
    
    print('Only using A(X) with flags == 0 if available, otherwise == 1')
    print('Setting uncertainty floor for A(X) to >= '+str(abundance_uncertainty_limit))
    
    converter = collections.OrderedDict()

    """
    before Karin ran combined
    converter['LiI']  = ['Li']
    converter['CI']   = ['C6588']
    converter['OI']   = ['O7772','O7774','O7775']
    converter['NaI']  = ['Na5683','Na5688'] # leaving out 'Na4752'
    converter['MgI']  = ['Mg4730','Mg5711','Mg7692'] # leaving out 'Mg7722', 'Mg7759', 'Mg7811'
    converter['AlI']  = ['Al6696','Al6699','Al7835','Al7836']
    converter['SiI']  = ['Si5684','Si5690','Si5701','Si5772','Si5793'] # leaving out 'Si5666','Si6722','Si7680'
    converter['KI']   = ['K5802','K7699']
    converter['CaI']  = ['Ca5857','Ca5868','Ca6494','Ca6500'] # leaving out Ca6509
    converter['ScI']  = ['Sc4744','Sc4753','Sc5672','Sc5687','Sc5724']
    converter['ScII'] = ['Sc5658','Sc5667','Sc5684','Sc6605']
    converter['TiI']  = ['Ti4758','Ti4759','Ti4778','Ti4782','Ti4798','Ti4802','Ti4820','Ti5689','Ti5716','Ti5720','Ti5739','Ti6717'] # 'Ti5866', 'Ti6599','Ti7853'
    converter['TiII'] = ['Ti4720','Ti4765','Ti4799','Ti4866','Ti4874'] #,'Ti4849'
    converter['VI']   = ['V4747','V4784','V4797','V4832']
    converter['CrI']  = ['Cr4775','Cr4789','Cr4801','Cr5702','Cr5720','Cr5788','Cr5845','Cr6630']
    converter['CrII'] = ['Cr4848']
    converter['MnI']  = ['Mn4739','Mn4762','Mn4766','Mn4783']
    #converter['FeI']  = ['Fe4789','Fe4793','Fe4794','Fe4803','Fe4808','Fe4876','Fe4890','Fe4891','Fe5651','Fe5652','Fe5661','Fe5663','Fe5679','Fe5680','Fe5696','Fe5702','Fe5705','Fe5731','Fe5732','Fe5742','Fe5775','Fe5778','Fe5807','Fe5809','Fe5812','Fe5815','Fe5850','Fe5853','Fe5855','Fe5859','Fe6482','Fe6495','Fe6499','Fe6518','Fe6546','Fe6593','Fe6594','Fe6598','Fe6609','Fe6628','Fe6648','Fe6678','Fe6699','Fe6704','Fe6714','Fe6725','Fe6733','Fe7710','Fe7723','Fe7748']
    #converter['FeII'] = ['Fe4720','Fe4731','Fe4833','Fe6516','Fe7712']
    converter['CoI']  = ['Co5647','Co6632','Co7713','Co7838']
    converter['NiI']  = ['Ni5847','Ni6586']
    converter['CuI']  = ['Cu5700','Cu5782']
    converter['ZnI']  = ['Zn4722','Zn4811']
    converter['RbI']  = ['Rb7800']
    converter['SrI']  = ['Sr6550']
    converter['YII']  = ['Y4855','Y4884','Y5663']
    converter['ZrI']  = ['Zr4739','Zr4772','Zr4806','Zr4828']
    converter['MoI']  = ['Mo5751','Mo5858']
    converter['RuI']  = ['Ru4869']
    converter['BaII'] = ['Ba5854','Ba6497']
    converter['LaII'] = ['La4716','La4749','La4804','La5806']
    converter['CeII'] = ['Ce4774']
    converter['NdII'] = ['Nd4811','Nd5741','Nd5770','Nd5812','Nd5842']
    converter['SmII'] = ['Sm4837','Sm4854']
    converter['EuII'] = ['Eu5819','Eu6645']
    converter['alpha'] = [
        'Mg4730','Mg5711','Mg7692',
        'Si5666','Si5684','Si5690','Si5701','Si5772','Si5793','Si6722','Si7680',
        'Ca5857','Ca5868','Ca6494','Ca6500',
        'Ti4758','Ti4759','Ti4778','Ti4782','Ti4798','Ti4802','Ti4820','Ti5689','Ti5716','Ti5720','Ti5739','Ti6717',
        'Ti4720','Ti4765','Ti4799','Ti4866','Ti4874'
    ]

    """
    
    converter['LiI']  = ['Li6708']
    converter['CI']   = ['C6588']
    converter['OI']   = ['O']
    converter['NaI']  = ['Na']
    converter['MgI']  = ['Mg5711']
    # run with old NLTE grid 
    # converter['SiI']  = ['Si5684','Si5690','Si5701','Si5772','Si5793'] # leaving out 'Si5666','Si6722','Si7680'
    converter['SiI']  = ['Si']
    converter['AlI']  = ['Al']
    converter['KI']   = ['K7699']
    # run with old NLTE grid 
    # converter['CaI']  = ['Ca5857','Ca5868','Ca6494','Ca6500'] # leaving out Ca6509
    converter['CaI']  = ['Ca']
    converter['ScI']  = ['Sc']# Sc4744','Sc4753','Sc5672','Sc5687','Sc5724']
    #converter['ScII'] = ['Sc5667','Sc5684','Sc6605']
    #converter['TiI']  = ['Ti4758','Ti4759','Ti4778','Ti4782','Ti4798','Ti4802','Ti4820','Ti5689','Ti5716','Ti5720','Ti5739','Ti6717'] # 'Ti5866', 'Ti6599','Ti7853'
    #converter['TiII'] = ['Ti4720','Ti4765','Ti4799','Ti4866','Ti4874'] #,'Ti4849'
    converter['TiI']  = ['Ti4758','Ti4759','Ti4782','Ti4802','Ti4820','Ti5739']
    converter['TiII']  = ['Ti4720','Ti4765','Ti4799','Ti4866']
    #converter['VI']   = ['V4747','V4784','V4797']
    converter['VI']   = ['V4797','V4832'] #'V4784',
    #converter['CrI']  = ['Cr4775','Cr4789','Cr4801','Cr5702','Cr5720','Cr5788','Cr5845','Cr6630']
    #converter['CrII'] = ['Cr4848']
    converter['CrI']  = ['Cr']
    converter['MnI']  = ['Mn']
    converter['CoI']  = ['Co5647','Co6490','Co6632','Co7713'] #maybe  Def NO 'Co4781','Co4900','Co6551','Co7838'
    converter['NiI']  = ['Ni5847','Ni6586']
    converter['CuI']  = ['Cu5700','Cu5782']
    converter['ZnI']  = ['Zn4722','Zn4811']
    converter['RbI']  = ['Rb7800']
    converter['SrI']  = ['Sr6550']
    converter['YII']  = ['Y4855','Y4884'] # leaving out 'Y4820','Y5663' 'Y5729'
    #converter['YII']  = ['Y']
    converter['ZrI']  = ['Zr4739','Zr4772','Zr4806','Zr4828','Zr5681']
    converter['MoI']  = ['Mo5858','Mo6619']#'Mo5689','Mo5751'
    converter['RuI']  = ['Ru4869','Ru5699']#'Ru4758'
    converter['BaII'] = ['Ba'] # not using 'Ba5854','Ba6497'
    converter['LaII'] = ['La4749','La4804','La5806'] # not using 'La4716',
    converter['CeII'] = ['Ce4774']
    converter['NdII'] = ['Nd4811','Nd5812'] # leaving out 'Nd5741','Nd5843','Nd6740','Nd5770','Nd5842'
    converter['SmII'] = ['Sm4720','Sm4848'] # leaving out 'Sm4792','Sm4837','Sm4854'
    converter['EuII'] = ['Eu6645'] # leaving out 'Eu5819',
    converter['alpha'] = [
        'Mg5711',
        'Si',
        'Ca',
        'Ti4758','Ti4759','Ti4782','Ti4802','Ti4820','Ti5739'
    ]
    # Until 191209
    #converter['alpha'] = [
    #    'Mg5711',
    #    'Si',
    #    'Ca',
    #    'Ti4758','Ti4759','Ti4778','Ti4782','Ti4798','Ti4802','Ti4820','Ti5689','Ti5716','Ti5720','Ti5739','Ti6717',
    #    'Ti4720','Ti4765','Ti4799','Ti4866','Ti4874'
    #]
    # Until 191105 (before using Karin's NLTE results)
    # converter['alpha'] = [
    #     'Mg5711',
    #     'Si5666','Si5684','Si5690','Si5701','Si5772','Si5793','Si6722','Si7680',
    #     'Ca5857','Ca5868','Ca6494','Ca6500',
    #     'Ti4758','Ti4759','Ti4778','Ti4782','Ti4798','Ti4802','Ti4820','Ti5689','Ti5716','Ti5720','Ti5739','Ti6717',
    #     'Ti4720','Ti4765','Ti4799','Ti4866','Ti4874'
    # ]
    # 'conservative_alpha'
    #converter['alpha'] = [
    #    'Mg4730','Mg5711',
    #    'Si5684','Si5690','Si5701','Si5772',
    #    'Ca5868',
    #    'Ti4758','Ti4759','Ti4778','Ti4782','Ti4798','Ti4802','Ti4820',
    #    'Ti4765','Ti4799','Ti4874'
    #]
    # Until 190519:
    #[
    #    'Mg4730','Mg5711','Mg7692',
    #    'Si5666','Si5684','Si5690','Si5701','Si5772','Si5793',
    #    'Ca5857','Ca5868','Ca6494','Ca6500',
    #    'Ti4758','Ti4759','Ti4778','Ti4782','Ti4798','Ti4802','Ti4820','Ti5689','Ti5716','Ti5720','Ti5739','Ti5866','Ti6599','Ti6717','Ti7853'
    #]
    
    """
    ii=where(mode eq 'Mg' or mode eq 'Si' or mode eq 'Ti')
    for i=0,n_elements(object)-1 do begin
       j=where(finite(res[i].abund[ii]) and res[i].aflag[ii] eq 0,jc)
       if jc ne 0 then begin
          res[i].alpha_fe   = total(res[i].abund[ii[j]]/res[i].e_abund[ii[j]]^2)/total(1./res[i].e_abund[ii[j]]^2)
          res[i].e_alpha_fe = sqrt(1./total(1./res[i].e_abund[ii[j]]^2))
          ;print,res[i].alpha_fe,res[i].e_alpha_fe,res[i].e_teff                                                                                                                                                
       endif
    endfor
    """
    
    def combine_line_measurements_for_element(each_element, final_output_data, final_output_abundances, data_index,debug=debug):

        """
        Combine the individual measurements
        
        Figure out which lines are not flagged
        
        1a) If more than 1 line is useful:
            Check for outliers with sigma-clipping
            For large sigma (>0.3, with exception of Li) or alpha: be more strict (-1.5,1.5)
            2a) If still more than 1 line is useful:
                simple error-weighted mean combination
            2b) elif 1 line useful (and element is not alpha_fe):
                give back that 1 line measurement
            2c) else:
                return nan value
         1b) elif 1 line useful (and element is not alpha_fe):
            give back that 1 line measurement
         1c) else: 
            return nan values
        """

        useful_line_bitmask = []
        useful_line_measurements = []
        useful_line_uncertainties = []

        if debug:
            print('Starting going through converter')
        
        for each_index, each_line in enumerate(converter[each_element]):

            if debug:
                print('index, line:',each_index,each_line)
            
            # if line measurement flag == 0
            if (
                (final_output_data['flag_'+each_line][data_index] == 0) &
                np.isfinite(final_output_abundances['A_'+each_line][data_index])
            ):

                useful_line_bitmask.append(2**each_index)
                useful_line_measurements.append(final_output_abundances['A_'+each_line][data_index] - abundance_zeropoints['A_'+each_line][0] - final_output_abundances['fe_h'][data_index])
                useful_line_uncertainties.append(final_output_data['cov_e_'+each_line][data_index])

        useful_line_bitmask=np.array(useful_line_bitmask)
        useful_line_measurements=np.array(useful_line_measurements)
        useful_line_uncertainties=np.array(useful_line_uncertainties)

        if debug:
            print('useful_line_bitmask',useful_line_bitmask)
            print('useful_line_measurements',useful_line_measurements)
            print('useful_line_uncertainties',useful_line_uncertainties)
        
        if len(useful_line_measurements) > 1:
            
            sigma_clipped_mean, sigma_clipped_median, sigma_clipped_std = sigma_clipped_stats(useful_line_measurements, sigma=clip_outlier_sigma)

            sigma_outliers = np.abs(sigma_clipped_mean - useful_line_measurements) > clip_outlier_sigma*sigma_clipped_std
            if ((sigma_clipped_std) > 0.3) | (each_element == 'alpha'): # Some outliers are causing problems
                if each_element != 'LiI':
                    more_robust = (useful_line_measurements > -1.5) & (useful_line_measurements < 2.5)
                    sigma_clipped_mean, sigma_clipped_median, sigma_clipped_std = sigma_clipped_stats(useful_line_measurements[more_robust], sigma=clip_outlier_sigma)

            if len(useful_line_measurements) > 1:
                return(
                    np.sum(useful_line_measurements[~sigma_outliers]/useful_line_uncertainties[~sigma_outliers]**2)/np.sum(1./useful_line_uncertainties[~sigma_outliers]**2),
                    np.sqrt(1./np.sum(1./useful_line_uncertainties[~sigma_outliers]**2)),
                    np.sum(useful_line_bitmask[~sigma_outliers]),
                    0)
                    #np.sum(useful_line_measurements[~sigma_outliers]/useful_line_uncertainties[~sigma_outliers]**2)/np.sum(1./useful_line_uncertainties[~sigma_outliers]**2),
                    #np.sqrt(1./np.sum(1./useful_line_uncertainties[~sigma_outliers]**2) + np.var(useful_line_measurements[~sigma_outliers])),
                    #np.sum(useful_line_bitmask[~sigma_outliers],
                    #0)
            elif (len(useful_line_measurements[~sigma_outliers]) == 1) & (each_element != 'alpha'):
                return(useful_line_measurements[~sigma_outliers],useful_line_uncertainties[~sigma_outliers],useful_line_bitmask[~sigma_outliers],0)
            else:
                return(np.nan,np.nan,np.nan,np.nan)
        elif (len(useful_line_measurements) == 1) & (each_element != 'alpha'):
            return(useful_line_measurements[0],useful_line_uncertainties[0],useful_line_bitmask[0],0)
        else:

            useful_line_bitmask = []
            useful_line_measurements = []
            useful_line_uncertainties = []
            
            if (
                (final_output_data['flag_'+each_line][data_index] == 1) &
                (final_output_abundances['flux_A_'+each_line][data_index] > 0.03) &
                (final_output_abundances['A_'+each_line][data_index] > -3)
            ):

                useful_line_bitmask.append(2**each_index)
                useful_line_measurements.append(final_output_abundances['A_'+each_line][data_index] - abundance_zeropoints['A_'+each_line][0] - final_output_abundances['fe_h'][data_index])
                useful_line_uncertainties.append(final_output_data['cov_e_'+each_line][data_index])

            useful_line_bitmask=np.array(useful_line_bitmask)
            useful_line_measurements=np.array(useful_line_measurements)
            useful_line_uncertainties=np.array(useful_line_uncertainties)

            if len(useful_line_measurements) >= 1:
                upper_limit = np.where(np.min(useful_line_measurements) == useful_line_measurements)[0][0]
                return(useful_line_measurements[upper_limit],useful_line_uncertainties[upper_limit],useful_line_bitmask[upper_limit],1)
            else:
                return(np.nan,np.nan,np.nan,np.nan)

    for each_element in converter.keys():
        (final_output_data[each_element+'_fe'], final_output_data['e_'+each_element+'_fe'], final_output_data['nr_'+each_element+'_fe'],  final_output_data['flag_'+each_element+'_fe']) = np.array([combine_line_measurements_for_element(each_element, final_output_data, final_output_abundances, data_index) for data_index in range(len(final_output_abundances['sobject_id']))]).T

    print('Uncertainty is calculated from sqrt(1/sum(1/unc**2)) only, but neglecting the line-by-line variance for now!')
        
    return(final_output_data, final_output_abundances)

In [9]:
def photometric_binarity_cool_outlier_flag(final_output_data):
    """
    This function loads isochrones for each [Fe/H] and checks 
    if the estimated Teff is cooler than the one for the oldest most metal-rich one
    
    The flag is useful to identify MS binaries with logg>4
    or unreliably cool stars for logg<4
    
    """

    try:
        isos = np.load('VAC_ages/Parsec_isochrones.npy')
    except:
        isos = np.load('VAC_ages/Parsec_isochrones.npy',fix_imports=True,allow_pickle=True,encoding='latin1')
        
    is_photometric_binary = np.zeros(len(final_output_data['fe_h']),dtype=bool)

    for each_iso_index, each_iso_feh in enumerate([isos[i].FeH for i in range(len(isos)-1)]):
        # We only employ this model for the non-metal-poor stars, because
        # for the metal-poor stars, we have already used the adjusted oldest isochrone
        if each_iso_feh >= -1.0:

            in_eiso_feh_bin = (final_output_data['fe_h'] > isos[each_iso_index].FeH) & (final_output_data['fe_h'] <= isos[each_iso_index+1].FeH) 
            if each_iso_index == 0:
                in_eiso_feh_bin = (final_output_data['fe_h'] <= each_iso_feh)
            if each_iso_index == len(isos)-1:
                in_eiso_feh_bin = (final_output_data['fe_h'] > each_iso_feh)

            # find the RGB tip of each isochrone by finding the first EEP below logg~2dex where the logg value of the next EEP is larger
            rgb_tip = np.where(
                (isos[each_iso_index].data[-1]['logG'][:-1] < 2) &
                (isos[each_iso_index].data[-1]['logG'][:-1] < isos[each_iso_index].data[-1]['logG'][1:]))[0][0]

            binary_identification_function = scipy.interpolate.interp1d(
                isos[each_iso_index].data[-1]['logG'][:rgb_tip],
                10**isos[each_iso_index].data[-1]['logTeff'][:rgb_tip],
                bounds_error=False,
                fill_value = (10**isos[each_iso_index].data[-1]['logTeff'][rgb_tip],10**isos[each_iso_index].data[-1]['logTeff'][0])
                #10**isos[each_iso_index].data[-1]['logTeff'][0],10**isos[each_iso_index].data[-1]['logTeff'][rgb_tip]]]
                )

            binary_identification_teff_lower = binary_identification_function(
                np.array(final_output_data['logg'][in_eiso_feh_bin]-0.15)
            )
            binary_identification_teff_upper = binary_identification_function(
                np.array(final_output_data['logg'][in_eiso_feh_bin]+0.15)
            )

            identified_binary_lower = final_output_data['teff'][in_eiso_feh_bin]+150 < binary_identification_teff_lower
            identified_binary_upper = final_output_data['teff'][in_eiso_feh_bin]+150 < binary_identification_teff_upper
            identified_binary = np.array(identified_binary_lower & identified_binary_upper)

            arange = np.arange(len(final_output_data['teff']))
            arange_identified_binary = (arange[in_eiso_feh_bin])[identified_binary]

            is_photometric_binary[arange_identified_binary] = True

    return is_photometric_binary

In [10]:
def cool_outlier_flag(final_output_data):
        
    # Here we read in a cutoff isochrone to decide which metal-poor stars are not reliable

    try:
        cutoff_isochrone = np.loadtxt('../input/MP_cutoff_isochrone.txt')
    except:

        isos = np.load('VAC_ages/Parsec_isochrones.npy')

        most_luminous_index = np.where(isos[0].data[-1]['logG'] == np.min(isos[0].data[-1]['logG']))[0]

        up_to_rgb_tip = np.arange(most_luminous_index[0])
        iso_change_1 = 28 # points from oldest, most metal-poor isochrone with teff-500 and logg-0.3
        iso_change_2 = 42 # points from oldest, most metal-poor isochrone with teff-500 and logg+0.3

        cutoff_isochrone = np.concatenate(([
            np.array([[10**isos[0].data[-1]['logTeff'][0]-1000,
             isos[0].data[-1]['logG'][0]]]),
            np.array([10**isos[0].data[-1]['logTeff'][up_to_rgb_tip[:iso_change_1]]-500,
             isos[0].data[-1]['logG'][up_to_rgb_tip[:iso_change_1]]-0.3]).T,
            np.array([10**isos[0].data[-1]['logTeff'][up_to_rgb_tip[iso_change_2:]]-500,
             isos[0].data[-1]['logG'][up_to_rgb_tip[iso_change_2:]]+0.3]).T,
            np.array([[10**isos[0].data[-1]['logTeff'][up_to_rgb_tip[-1]]-500,
             isos[0].data[-1]['logG'][up_to_rgb_tip[-1]]-1.3]])
            ]))

        np.savetxt('../input/MP_cutoff_isochrone.txt',cutoff_isochrone,fmt='%s')

    cutoff_isochrone_function = scipy.interpolate.interp1d(cutoff_isochrone[:,1],cutoff_isochrone[:,0])
    cutoff_teff = cutoff_isochrone_function(np.array(final_output_data['logg']))

    cool_outlier = photometric_binarity_cool_outlier_flag(final_output_data)
    
    
    unreliable_cool_mp_stars = (
        ((final_output_data['teff'] < cutoff_teff) & (final_output_data['fe_h'] < -1.2)) |
        (cool_outlier & (final_output_data['logg'] < 4.0))
    )
    
    return(unreliable_cool_mp_stars)

In [11]:
def apply_sp_flags(final_output_data, final_output_abundances):
    
    get_bin = lambda x, n: format(int(x), 'b').zfill(n)
    
    sme_idl_bitmask = np.array([get_bin(x, 10) for x in final_output_data['flag_sp']])
    #  0 : alright  
    #+ 1 : convergence == non-finite SPs  
    #+ 2 : grid limit reached  
    #+ 4 : Gaussian RV fit failed  
    #+ 8 : ELLI mass estimated failed  
    #+16 : Timeout on ISAAC
        
    red_bitmask = np.array([get_bin(x, 10) for x in final_output_data['red_flag']])
    #  0 : for no flags,
    #+ 1 : for bad wavelength solution in ccd_1,
    #+ 2 : for bad wavelength solution in ccd_2, 
    #+ 4 : for bad wavelength solution in ccd_3,
    #+ 8 : for bad wavelength solution in ccd_4,
    #+16 : for molecfit fail in ccd_3,
    #+32 : for molecfit fail in ccd_4,
    #+64 : if the object is actually a twilight flat.
    
    # tsne_old is a combination of tsne_class_1_0.fits and tsne_classification_dr52_2018_04_09.csv
    tsne_old = Table.read('../input/tSNE/tsne_flags_combined.fits')

    # CMP giants
    # HaHb emission
    # binary
    # hot stars
    # mol. abs. bands
    # problematic - ccd2 continuum
    # problematic - ccd3 continuum
    # problematic - ccd3 spikes
    # problematic - ccd4
    # problematic - ccd4 oversubtraction
    # problematic - ccd4 spikes
    # problematic - ccd4 strong spike
    # problematic - neg. flux
    # problematic - osc. cont.

    # Dictionary: 
    #
    # BIN - Binary
    # HAE - Halpha emission
    # HBE - Hbeta emission
    # WRD - weird (very general)
    # MAB - Molecular absorption bands
    # CMP - Cool metal poor giants
    # HOT - Hot stars:)
    # HFR - Hot fast rotators
    # TAB - Telluric absorption
    # TEM - Telluric emission
    # SPI - Spikes
    # NTR - Nothing to report
    # CEH - Carbon-enhanced (SWAN bands)
    #

    binary_old = (
        (tsne_old['class_norm'] == 'binary') |
        (tsne_old['class_red'] == 'binary') |
        (tsne_old['tsne'] == 'BIN')
    )
    emission_old = (
        (tsne_old['class_red'] == 'HaHb emission') |
        (tsne_old['class_norm'] == 'HaHb emission') |
        (tsne_old['tsne'] == 'HAE') |
        (tsne_old['tsne'] == 'HBE')
        )
    reduction_old = (
        (tsne_old['class_red']  == 'problematic') |
        (tsne_old['class_norm'] == 'problematic - ccd2 continuum') |
        (tsne_old['class_norm'] == 'problematic - ccd3 continuum') |
        (tsne_old['class_norm'] == 'problematic - ccd3 spikes') |
        (tsne_old['class_norm'] == 'problematic - ccd4') |
        (tsne_old['class_norm'] == 'problematic - ccd4 oversubtraction') |
        (tsne_old['class_norm'] == 'problematic - ccd4 spikes') |
        (tsne_old['class_norm'] == 'problematic - ccd4 strong spike') |
        (tsne_old['class_norm'] == 'problematic - neg. flux') |
        (tsne_old['class_norm'] == 'problematic - osc. cont.') |
        (tsne_old['tsne'] == 'WRD') |
        (tsne_old['tsne'] == 'TAB') |
        (tsne_old['tsne'] == 'TEM') |
        (tsne_old['tsne'] == 'SPI')
        )

    tsne_new = Table.read('../input/tSNE/tsne_classification_dr53_2018_12_30.csv')
    bins      = (tsne_new['classification'] == 'BIN')
    emission = (
                (tsne_new['classification'] == 'HAE') | 
                (tsne_new['classification'] == 'HBE')
                )
    reduction = (
                (tsne_new['classification'] == 'WRD') | 
                (tsne_new['classification'] == 'TAB') | 
                (tsne_new['classification'] == 'TEM') | 
                (tsne_new['classification'] == 'SPI')
                )
    
    tsne_hot = Table.read('../input/tSNE/hot_stars_binary_emission_2020_02_07.csv')
    
    bins_hot  = (tsne_hot['classification'] == 'BIN')
    emission_hot = (tsne_hot['classification'] == 'EMISSION')

    np.savetxt('../input/tsne_binaries.txt', np.unique(np.concatenate((np.array(tsne_old['sobject_id'][binary_old]),np.array(tsne_new['sobject_id'][bins]),np.array(tsne_hot['sobject_id'][bins_hot])))), fmt='%s')
    tsne_binary  = np.loadtxt('../input/tSNE/tsne_binaries.txt',dtype=np.int)

    np.savetxt('../input/tsne_emission.txt', np.unique(np.concatenate((np.array(tsne_old['sobject_id'][emission_old]),np.array(tsne_new['sobject_id'][emission]),np.array(tsne_hot['sobject_id'][emission_hot])))), fmt='%s')
    tsne_emission  = np.loadtxt('../input/tSNE/tsne_emission.txt',dtype=np.int)

    np.savetxt('../input/tsne_reduction_issues.txt', np.unique(np.concatenate((np.array(tsne_old['sobject_id'][reduction_old]),np.array(tsne_new['sobject_id'][reduction])))), fmt='%s')
    tsne_reduction = np.loadtxt('../input/tSNE/tsne_reduction_issues.txt',dtype=np.int)
    
    bitmask_sp = np.zeros(len(final_output_data['sobject_id']),dtype=np.int)
    
    print('Applying the following flags:')
    
    def raise_bitmask(bit, position):
        #print(position)
        bitmask_sp[position] += bit
    
    # Raise bitmask 1
    print('   1: RUWE > 1.4 (bad astrometric solution)')
    raise_bitmask(1, final_output_data['ruwe'] > 1.4)
    
    # Raise bitmask 2
    print('   2: unreliable broadening')
    raise_bitmask(2, (
        (final_output_data['vbroad'] <= 3.) |
        (final_output_data['vbroad'] >= 100.))
        )
    
    # Raise bitmask 4
    print('   4: Low S/N (below 5 or 10? below this value for all CCDs or just CCD2 as SNR-tracer?)')
    raise_bitmask(4, final_output_data['snr_c2_iraf'] <= 10)

    # Raise bitmask 8
    print('   8: reduction issue:')
    print('      a) Wavelength solution (propagating of red_flag)')
    print('      b) t-SNE projected reduction issues: Negative/positive fluxes, spikes, etc.')
    raise_bitmask(8, np.array([(x[-1]=='1') | (x[-2]=='1') | (x[-3]=='1') | (x[-4]=='1') | (y in tsne_reduction)  for (x, y) in zip(red_bitmask, final_output_data['sobject_id'])]))
    
    # Raise bitmask 16
    print('  16: t-SNE projected emission features')
    raise_bitmask(16, np.array([each in tsne_emission for each in final_output_data['sobject_id']]))

    # Raise bitmask 32
    print('  32: t-SNE projected binaries')
    raise_bitmask(32, np.array([each in tsne_binary for each in final_output_data['sobject_id']]))
    
    # Raise bitmask 64
    print('  64: Photometric binarity flag')
    is_photometric_binary = photometric_binarity_cool_outlier_flag(final_output_data)
    raise_bitmask(64, ((is_photometric_binary == True) & (final_output_data['logg'] >= 4.0)))

    # Raise bitmask 128
    print(' 128: SNR-dependent high SME chi2 (bad fit) / FYI: median chi2_sp is 0.748')
    raise_bitmask(128, final_output_data['chi2_sp'] > np.exp(0.08*final_output_data['snr_c2_iraf'])+0.1*final_output_data['snr_c2_iraf'])
    
    """
    Karin's suggestion:
    if not using chi2_limit(snr, teff), go only for selecting the definite outliers
    raise_bitmask(16, final_output_data['chi2_sp'] > np.exp(0.08*final_output_data['snr_c2_iraf'])+0.1*final_output_data['snr_c2_iraf'])

    My suggestion:
    raise_bitmask(16, final_output_data['chi2_sp'] / 0.35 > 3. * np.exp(1/90.*final_output_data['snr_c2_iraf']))
    which cuts away most cool giants...

    Morgan's suggestion:
    (chi2_sp - (0.75 - 1) / 10 * ln(snr_c2_iraf)) / (((.005)/60.) * snr_c2_iraf * snr_c2_iraf + 0.3) > 3. * chi2_sp
    """
    
    """
    t-SNE flags as taken from "tsne_classification_dr52_2018_04_09.csv"
    TO BE UPDATED WITH TRAVEN'S NEW DR5.3 FLAGS
    """
    
    # Raise bitmask 256
    print(' 256: Problems with Fe lines, where lineflux is not between 0.03 and 1.00')
    raise_bitmask(256,
        (
        (final_output_abundances['flux_A_Fe'] <= 0.03) | 
        (final_output_abundances['flux_A_Fe'] > 1.00)
        ) |
        np.isnan(final_output_data['fe_h']) |
        (final_output_data['fe_h'] > 1.0)
        )
    
    # Raise bitmask 512')
    print(' 512: sme did not finish')
    print('      a) no convergence == non-finite SPs')
    print('      b) Gaussian RV fit failed')
    print('      c) Timeout on ISAAC')
    raise_bitmask(512, np.array([((x[-1]=='1') | (x[-3]=='1') | (x[-4]=='1') |(x[-5]=='1') ) for x in sme_idl_bitmask]))

    unreliable_cool_mp_stars = cool_outlier_flag(final_output_data)

    # Raise bitmask 1024
    print('1024: MARCS grid limit reached or outside of reasonable parameter range')
    raise_bitmask(1024, (
        np.array([x[-2]=='1' for x in sme_idl_bitmask]) | 
        (final_output_data['fe_h_atmo'] > 1.0) |
        (final_output_data['fe_h'] > 1.0) |
        (final_output_data['teff'] <=3000) |
        # This is the clump of metal-poor stars below Teff~4000 K
        ((final_output_data['teff'] <=4000) & (final_output_data['fe_h'] <=-1.0)) |
        # The aforementioned unreliable cool MP stars
        (unreliable_cool_mp_stars == True)
        )
    )
    adjust_fe_h = (final_output_data['fe_h'] >= 1.0)
    final_output_data['fe_h'][adjust_fe_h] = 1.0
    adjust_fe_h = (final_output_data['teff'] < 3000.)
    final_output_data['teff'][adjust_fe_h] = 3000.

    final_output_data['flag_sp'] = bitmask_sp
    
    return(final_output_data, final_output_abundances)

In [12]:
def apply_ab_flags(final_output_data, final_output_abundances):

    get_bin = lambda x, n: format(int(x), 'b').zfill(n)   

    for each_key in final_output_abundances.keys():

        if each_key[0:2]=='A_':
                        
            each_element = each_key[2:]

            if each_element == 'Fe':
                element = 'fe_h'

                print('Applying the following flags:')
                print(' 1: Upper limit')
                print(' 2: Bad chi2 fit')
                print(' 4: Saturation')
                print(' 8: Bad wavelength solution / rv for Li6708')
                print('16: Bad stellar parameter flag (>= 128)')
                print('32: No abundance value at all')

            else:
                element = each_element

            old_bitmask = np.array([get_bin(x, 10) for x in final_output_data['flag_'+element]])

            bitmask_element = np.zeros(len(final_output_data['sobject_id']),dtype=np.int)

            def raise_bitmask(bit, position):
                #print(position)
                bitmask_element[position] += bit

            # Raise bitmask 1
            # - upper limit
            raise_bitmask(1, np.array([x[-1]=='1' for x in old_bitmask]))
            
            # Raise bitmask 2
            # - bad chi2 fit
            raise_bitmask(2, np.array([x[-2]=='1' for x in old_bitmask]))

            # Raise bitmask 4
            # - saturation
            raise_bitmask(4, np.where(
                (final_output_data['cov_e_'+element] >= 0.3) | 
                (final_output_data['cov_e_'+element] >= 3./final_output_data['snr_c2_iraf'] + 0.01)
                )
            )

            if element == 'Li':
                # Raise bitmask 8
                # - bad wavelength solution / rv for Li6708
                # - cool dwarfs with subsolar A(Li)
                raise_bitmask(8, 
                    (
                      (np.abs(final_output_data['rv_6708']) > 10) |
                      (
                          (final_output_data['teff'] < 4500) & 
                          (final_output_data['logg'] > 4.0) & 
                          (final_output_data['Li6708_fe'] + final_output_data['fe_h'] < 0.0) 
                      )
                    )
                )
            
            # Raise bitmask 16
            # - flag_sp above 128
            raise_bitmask(16, np.where(
                (final_output_data['flag_sp'] >= 128)
                )
            )
            
            if element != 'fe_h':
                # Raise bitmask 32
                # - no abundance value at all
                raise_bitmask(32, np.isnan(final_output_abundances['A_'+element]))

            final_output_data['flag_'+element] = bitmask_element

    return(final_output_data, final_output_abundances)

In [13]:
def adjust_unreliable_upper_limits(final_output_data):
    """
    These adjustments are based on outcome of 
    GALAH_DR3/validation/upper_limits/dr3_upper_limit_validation.ipynb
    
    subgroups = dict()
    subgroups['Coolest_MS'] = np.array([3000,4500, 4.00,5.25])
    subgroups['Cool_MS']    = np.array([4500,5500, 4.00,5.25])
    subgroups['Warm_MS']    = np.array([5500,6600, 3.50,5.25])
    subgroups['Hot_MS']     = np.array([6600,8000, 3.50,5.25])
    subgroups['Hot_Lum1']   = np.array([5500,6600,-0.50,3.50])
    subgroups['Hot_Lum2']   = np.array([6600,8000,-0.50,3.50])
    subgroups['Subgiants']  = np.array([4250,5500, 3.50,4.00])
    subgroups['RGB_tip']    = np.array([4250,5500, 3.00,3.50])
    subgroups['1st_RC']     = np.array([4250,5500, 2.20,2.60])
    subgroups['RGB_L1']     = np.array([4250,5500, -0.5,2.20])
    subgroups['RGB_L2']     = np.array([4000,4250, -0.5,2.20])
    subgroups['RGB_L3']     = np.array([3750,4000, -0.5,2.20])
    subgroups['RGB_L4']     = np.array([3000,3750, -0.5,2.20])
    subgroups['Bump']       = np.array([4250,4900, 2.60,3.00])
    subgroups['2nd_RC']     = np.array([4900,5500, 2.60,3.00])
    subgroups['PMS']       = np.array([3000,4250, 2.20,4.00])
    """
    
    def apply_upper_limit(element, adjust_upper_limit):
        adjust_upper_limit_plus = (
            adjust_upper_limit &
            bitmask.bitfield_to_boolean_mask(np.array(final_output_data['flag_'+element+'_fe'],dtype=int))
        )
        final_output_data[element+'_fe'][adjust_upper_limit_plus] = np.nan
        final_output_data['e_'+element+'_fe'][adjust_upper_limit_plus] = np.nan
        final_output_data['flag_'+element+'_fe'][adjust_upper_limit_plus] += 32
    
    #Li
    #Adjusted flag_li_fe for subsolar A(Li) for Coolest_MS
    
    #C
    adjust_upper_limit = (
        ~((final_output_data['teff'] > 5250) | ((final_output_data['logg'] > 3) & (final_output_data['teff'] > 5000)))
    )
    apply_upper_limit('CI',adjust_upper_limit)
    
    #O
    adjust_upper_limit = (
        ~((final_output_data['teff'] > 4250) | (final_output_data['logg'] > 2.5))
    )
    apply_upper_limit('OI',adjust_upper_limit)

    #Na
    adjust_upper_limit = (
        ~((final_output_data['teff'] < 6600))
    )
    apply_upper_limit('NaI',adjust_upper_limit)
    
    #Mg
    adjust_upper_limit = (
        ~((final_output_data['teff'] < 6600) & (final_output_data['MgI_fe'] < -final_output_data['fe_h'] - 0.5))
    )
    apply_upper_limit('MgI',adjust_upper_limit)
    
    #Al
    adjust_upper_limit = (
        ~(final_output_data['teff'] < 6600)
    )
    apply_upper_limit('AlI',adjust_upper_limit)
   
    #Si
    adjust_upper_limit = (
        ~((final_output_data['teff'] > 4250) & (final_output_data['teff'] < 6600) & (final_output_data['SiI_fe'] < -0.9*final_output_data['fe_h'] - 0.5))
    )
    apply_upper_limit('SiI',adjust_upper_limit)
    
    #K
    adjust_upper_limit = (
        ~(np.isfinite(final_output_data['KI_fe']))
    )
    apply_upper_limit('KI',adjust_upper_limit)

    #Sc
    adjust_upper_limit = (
        ~((final_output_data['teff'] > 4250) & (final_output_data['teff'] < 6600) & (final_output_data['ScI_fe'] < -final_output_data['fe_h'] - 0.5))
    )
    apply_upper_limit('ScI',adjust_upper_limit)
    
    #Ti
    adjust_upper_limit = (
        ~(np.isfinite(final_output_data['TiI_fe']))
    )
    apply_upper_limit('TiI',adjust_upper_limit)

    #Ti2
    adjust_upper_limit = (
        ~(np.isfinite(final_output_data['TiII_fe']))
    )
    apply_upper_limit('TiII',adjust_upper_limit)

    #V
    adjust_upper_limit = (
        ~((final_output_data['teff'] > 4250) & (final_output_data['teff'] < 5500) & (final_output_data['logg'] < 3.5) & (final_output_data['VI_fe'] < -final_output_data['fe_h']))
    )
    apply_upper_limit('VI',adjust_upper_limit)

    print('CHECK Cr again! The galah_collect might have messed up the lineflux -> Cr6630 weak, thus causing low [Cr/Fe] upper limits?!')
    #Cr
    adjust_upper_limit = (
        ~((final_output_data['teff'] > 4250) & (final_output_data['teff'] < 6600) & (final_output_data['CrI_fe'] < -final_output_data['fe_h'] - 0.5))
    )
    apply_upper_limit('CrI',adjust_upper_limit)

    #Mn
    adjust_upper_limit = (
        ~((final_output_data['teff'] > 4250) & (final_output_data['teff'] < 6600) & ((final_output_data['fe_h'] < -0.75) & (final_output_data['MnI_fe'] < -final_output_data['fe_h'] - 0.5)))
    )
    apply_upper_limit('MnI',adjust_upper_limit)

    #Ni
    adjust_upper_limit = (
        ~((final_output_data['teff'] > 4250) & (final_output_data['teff'] < 6600) & (final_output_data['NiI_fe'] < -final_output_data['fe_h'] - 0.5))
    )
    apply_upper_limit('NiI',adjust_upper_limit)

    #Cu
    adjust_upper_limit = (
        ~((final_output_data['teff'] > 4250) & (final_output_data['teff'] < 6600) & (final_output_data['CuI_fe'] < -final_output_data['fe_h'] - 0.5))
    )
    apply_upper_limit('CuI',adjust_upper_limit)

    #Zn
    adjust_upper_limit = (
        ~((final_output_data['teff'] > 4250) & 
          (final_output_data['teff'] < 6600) & 
          # MP or MSTO star (neither giant nor subgiant)
          (
              (final_output_data['teff'] > 5500) |
              (final_output_data['logg'] > 4.0) |
              (final_output_data['fe_h'] < -0.75)
          ) & 
          (final_output_data['ZnI_fe'] < -final_output_data['fe_h'] + 0.5))
    )
    apply_upper_limit('ZnI',adjust_upper_limit)

    #Rb
    adjust_upper_limit = (
        ~((final_output_data['teff'] > 4250) & (final_output_data['teff'] < 5500))
    )
    apply_upper_limit('RbI',adjust_upper_limit)

    #Sr
    adjust_upper_limit = (
        ~((final_output_data['teff'] > 4250) & 
          (final_output_data['teff'] < 6600) & 
          (final_output_data['SrI_fe'] < -final_output_data['fe_h'] + 0.5))
    )
    apply_upper_limit('SrI',adjust_upper_limit)

    #Y
    adjust_upper_limit = (
        ~((final_output_data['teff'] > 4250) & 
          (final_output_data['teff'] < 6600) & 
          # dwarf
          (
              (final_output_data['teff'] > 5500) |
              (final_output_data['logg'] > 4.0)
          ) & 
          (final_output_data['YII_fe'] < -final_output_data['fe_h'])
         )
    )
    apply_upper_limit('YII',adjust_upper_limit)

    #Zr
    adjust_upper_limit = (
        ~(final_output_data['teff'] < 4250)
    )
    apply_upper_limit('ZrI',adjust_upper_limit)

    #Mo
    adjust_upper_limit = (
        ~((final_output_data['teff'] < 4250))
    )
    apply_upper_limit('MoI',adjust_upper_limit)

    #Ru
    adjust_upper_limit = (
        ~((final_output_data['teff'] < 4250))
    )
    apply_upper_limit('RuI',adjust_upper_limit)
    
    #Ba
    adjust_upper_limit = (
        ~((final_output_data['teff'] < 6600))
    )
    apply_upper_limit('BaII',adjust_upper_limit)

    #La
    adjust_upper_limit = (
        # only giants and subdwarfs
        ~((final_output_data['teff'] < 5500) & (final_output_data['logg'] < 4.0) )
    )
    apply_upper_limit('LaII',adjust_upper_limit)

    #Ce
    adjust_upper_limit = (
        # only cooler than 5500 K
        ~((final_output_data['teff'] < 5500))
    )
    apply_upper_limit('CeII',adjust_upper_limit)

    #Nd
    adjust_upper_limit = (
        # only giants
        ~((final_output_data['teff'] < 5500) & (final_output_data['logg'] < 3.5) )
    )
    apply_upper_limit('NdII',adjust_upper_limit)

    #Sm
    adjust_upper_limit = (
        # only giants and subdwarfs
        ~((final_output_data['teff'] < 5500) & (final_output_data['logg'] < 4.0) )
    )
    apply_upper_limit('SmII',adjust_upper_limit)


    #Eu
    adjust_upper_limit = (
        # only giants and subdwarfs
        ~((final_output_data['teff'] < 5500) & (final_output_data['logg'] < 4.0) )
    )
    apply_upper_limit('EuII',adjust_upper_limit)




    return(final_output_data)

In [14]:
def write_to_fits(final_output_data, final_output_abundances,output_filename):
    output_pandas = pandas.DataFrame(final_output_data,columns=final_output_data.keys())
    output_astropy = Table.from_pandas(output_pandas)

    output_astropy.write('processed_files/GALAH_iDR3_'+output_filename+'.fits',overwrite=True)

    output_a_pandas = pandas.DataFrame(final_output_abundances,columns=final_output_abundances.keys())
    output_a_astropy = Table.from_pandas(output_a_pandas)

    output_a_astropy.write('processed_files/GALAH_iDR3_'+output_filename+'_abund.fits',overwrite=True)

    return output_astropy, output_a_astropy

## Execute

In [15]:
product_list = []
for each_subset in np.arange(0,66):
        if np.shape(glob.glob('sme_result_files/GALAH_10k_'+str(each_subset)+'_lbol_final.fits'))[0] > 0:
            product_list.append('10k_'+str(each_subset))
product_list = np.array(product_list)

#product_list = ['10k_43']

# product_list = [
#      '180602',
# ]

#product_list = ['GBS','seis','OpenClusters','GlobularClusters','random10000','ts_DR2','high_vtot','Li_rich_giants']

print(product_list,len(product_list))

['10k_0' '10k_1' '10k_2' '10k_3' '10k_4' '10k_5' '10k_6' '10k_7' '10k_8'
 '10k_9' '10k_10' '10k_11' '10k_12' '10k_13' '10k_14' '10k_15' '10k_16'
 '10k_17' '10k_18' '10k_19' '10k_20' '10k_21' '10k_22' '10k_23' '10k_24'
 '10k_25' '10k_26' '10k_27' '10k_28' '10k_29' '10k_30' '10k_31' '10k_32'
 '10k_33' '10k_34' '10k_35' '10k_36' '10k_37' '10k_38' '10k_39' '10k_40'
 '10k_41' '10k_42' '10k_43' '10k_44' '10k_45' '10k_46' '10k_47' '10k_48'
 '10k_49' '10k_50' '10k_51' '10k_52' '10k_53' '10k_54' '10k_55' '10k_56'
 '10k_57' '10k_58' '10k_59' '10k_60' '10k_61' '10k_62' '10k_63' '10k_64'
 '10k_65'] 66


In [16]:
# for product_name in product_list:
#     #create_data_product(product_name)
#     print(product_name)
    
#     product_subsets, product_input_data_path, product_pipeline = get_product_information(product_name)

#     dr3_output_structure, sme_data, iraf_data, abundance_zeropoints = get_input_data(
#         product_subsets,
#         product_input_data_path,
#         product_pipeline)

#     (final_output_data, final_output_abundances) = combine_SME_IRAF_to_FINAL(
#         output_filename = product_name, 
#         product_pipeline=product_pipeline,
#         sme_data=sme_data,
#         iraf_data=iraf_data,
#         dr3_output_structure=dr3_output_structure,
#         abundance_zeropoints=abundance_zeropoints);

#     for each_key in ['teff','vbroad','fe_h','fe_h_atmo','rv_galah']:
#         final_output_data = compute_final_uncertainty(final_output_data, each_key)

#     # MC sample the uncertainties for logg and compute final logg uncertainty
#     final_output_data['cov_e_logg'] = compute_logg_uncertainty(final_output_data)
#     final_output_data = compute_final_uncertainty(final_output_data, 'logg')

#     (final_output_data, final_output_abundances) = apply_sp_flags(final_output_data, final_output_abundances)

#     (final_output_data, final_output_abundances) = apply_ab_flags(final_output_data, final_output_abundances)

#     (final_output_data, final_output_abundances) = compute_final_abundance_uncertainty(final_output_data, final_output_abundances)
    
#     (final_output_data,final_output_abundances) =  combine_line_by_line(final_output_data, final_output_abundances, abundance_zeropoints)

#     final_output_data =  adjust_unreliable_upper_limits(final_output_data)

#     fits_data = write_to_fits(
#         final_output_data,
#         final_output_abundances,
#         output_filename = product_name)

In [17]:
def create_data_product(product_name):

    print(product_name)
    
    product_subsets, product_input_data_path, product_pipeline = get_product_information(product_name)

    dr3_output_structure, sme_data, iraf_data, abundance_zeropoints = get_input_data(
        product_subsets,
        product_input_data_path,
        product_pipeline)

    (final_output_data, final_output_abundances) = combine_SME_IRAF_to_FINAL(
        output_filename = product_name, 
        product_pipeline=product_pipeline,
        sme_data=sme_data,
        iraf_data=iraf_data,
        dr3_output_structure=dr3_output_structure,
        abundance_zeropoints=abundance_zeropoints);

    for each_key in ['teff','vbroad','fe_h','fe_h_atmo','rv_galah']:
        final_output_data = compute_final_uncertainty(final_output_data, each_key)

    # MC sample the uncertainties for logg and compute final logg uncertainty
    final_output_data['cov_e_logg'] = compute_logg_uncertainty(final_output_data)
    final_output_data = compute_final_uncertainty(final_output_data, 'logg')

    (final_output_data, final_output_abundances) = apply_sp_flags(final_output_data, final_output_abundances)

    (final_output_data, final_output_abundances) = apply_ab_flags(final_output_data, final_output_abundances)

    (final_output_data, final_output_abundances) = compute_final_abundance_uncertainty(final_output_data, final_output_abundances)
    
    (final_output_data,final_output_abundances) =  combine_line_by_line(final_output_data, final_output_abundances, abundance_zeropoints)

    final_output_data =  adjust_unreliable_upper_limits(final_output_data)

    fits_data = write_to_fits(
        final_output_data,
        final_output_abundances,
        output_filename = product_name)

In [18]:
multi = True

if multi == True:
    import multiprocessing
    from multiprocessing import Pool

    if __name__ == '__main__':
        with Pool(3) as p:
            p.map(create_data_product, product_list)
else:
    for product_name in product_list:
        create_data_product(product_name)

10k_6
10k_0
10k_12
Got input data
Combining information
Got input data
Got input data
Combining information
Combining information
NB: We are leaving out logg for now, because we compute its final uncertainty later
NB: We are leaving out logg for now, because we compute its final uncertainty later
NB: We are leaving out logg for now, because we compute its final uncertainty later
NB: We are leaving out logg for now, because we compute its final uncertainty later
NB: We are leaving out logg for now, because we compute its final uncertainty later
Sampling logg uncertainty
NB: We are leaving out logg for now, because we compute its final uncertainty later
NB: We are leaving out logg for now, because we compute its final uncertainty later
NB: We are leaving out logg for now, because we compute its final uncertainty later
NB: We are leaving out logg for now, because we compute its final uncertainty later
NB: We are leaving out logg for now, because we compute its final uncertainty later
NB: 

/Users/svenbuder/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: RuntimeWarning: Mean of empty slice
/Users/svenbuder/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: RuntimeWarning: Mean of empty slice
/Users/svenbuder/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: RuntimeWarning: Mean of empty slice
/Users/svenbuder/anaconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/Users/svenbuder/anaconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/Users/svenbuder/anaconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


Done sampling logg uncertainty
Done sampling logg uncertainty
Done sampling logg uncertainty
NB: We are leaving out logg for now, because we compute its final uncertainty later
NB: We are leaving out logg for now, because we compute its final uncertainty later
NB: We are leaving out logg for now, because we compute its final uncertainty later
Applying the following flags:
   1: RUWE > 1.4 (bad astrometric solution)
   2: unreliable broadening
   4: Low S/N (below 5 or 10? below this value for all CCDs or just CCD2 as SNR-tracer?)
   8: reduction issue:
      a) Wavelength solution (propagating of red_flag)
      b) t-SNE projected reduction issues: Negative/positive fluxes, spikes, etc.
  16: t-SNE projected emission features
Applying the following flags:
   1: RUWE > 1.4 (bad astrometric solution)
   2: unreliable broadening
   4: Low S/N (below 5 or 10? below this value for all CCDs or just CCD2 as SNR-tracer?)
   8: reduction issue:
      a) Wavelength solution (propagating of red_f

/Users/svenbuder/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: RuntimeWarning: Mean of empty slice
/Users/svenbuder/anaconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


Done sampling logg uncertainty
NB: We are leaving out logg for now, because we compute its final uncertainty later
Applying the following flags:
   1: RUWE > 1.4 (bad astrometric solution)
   2: unreliable broadening
   4: Low S/N (below 5 or 10? below this value for all CCDs or just CCD2 as SNR-tracer?)
   8: reduction issue:
      a) Wavelength solution (propagating of red_flag)
      b) t-SNE projected reduction issues: Negative/positive fluxes, spikes, etc.
  16: t-SNE projected emission features


...

Done sampling logg uncertainty
NB: We are leaving out logg for now, because we compute its final uncertainty later
Applying the following flags:
   1: RUWE > 1.4 (bad astrometric solution)
   2: unreliable broadening
   4: Low S/N (below 5 or 10? below this value for all CCDs or just CCD2 as SNR-tracer?)
   8: reduction issue:
      a) Wavelength solution (propagating of red_flag)
      b) t-SNE projected reduction issues: Negative/positive fluxes, spikes, etc.
  16: t-SNE projected emission features
  32: t-SNE projected binaries
  64: Photometric binarity flag
 128: SNR-dependent high SME chi2 (bad fit) / FYI: median chi2_sp is 0.748
 256: Problems with Fe lines, where lineflux is not between 0.03 and 1.00
 512: sme did not finish
      a) no convergence == non-finite SPs
      b) Gaussian RV fit failed
      c) Timeout on ISAAC
1024: MARCS grid limit reached or outside of reasonable parameter range
Applying the following flags:
 1: Upper limit
 2: Bad chi2 fit
 4: Saturation
 8: Bad

/Users/svenbuder/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: RuntimeWarning: Mean of empty slice
/Users/svenbuder/anaconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


Done sampling logg uncertainty
NB: We are leaving out logg for now, because we compute its final uncertainty later
Applying the following flags:
   1: RUWE > 1.4 (bad astrometric solution)
   2: unreliable broadening
   4: Low S/N (below 5 or 10? below this value for all CCDs or just CCD2 as SNR-tracer?)
   8: reduction issue:
      a) Wavelength solution (propagating of red_flag)
      b) t-SNE projected reduction issues: Negative/positive fluxes, spikes, etc.
  16: t-SNE projected emission features
  32: t-SNE projected binaries
  64: Photometric binarity flag
 128: SNR-dependent high SME chi2 (bad fit) / FYI: median chi2_sp is 0.748
 256: Problems with Fe lines, where lineflux is not between 0.03 and 1.00
 512: sme did not finish
      a) no convergence == non-finite SPs
      b) Gaussian RV fit failed
      c) Timeout on ISAAC
1024: MARCS grid limit reached or outside of reasonable parameter range
Applying the following flags:
 1: Upper limit
 2: Bad chi2 fit
 4: Saturation
 8: Bad

/Users/svenbuder/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: RuntimeWarning: Mean of empty slice
/Users/svenbuder/anaconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


Done sampling logg uncertainty
NB: We are leaving out logg for now, because we compute its final uncertainty later
Applying the following flags:
   1: RUWE > 1.4 (bad astrometric solution)
   2: unreliable broadening
   4: Low S/N (below 5 or 10? below this value for all CCDs or just CCD2 as SNR-tracer?)
   8: reduction issue:
      a) Wavelength solution (propagating of red_flag)
      b) t-SNE projected reduction issues: Negative/positive fluxes, spikes, etc.
  16: t-SNE projected emission features
  32: t-SNE projected binaries
  64: Photometric binarity flag
 128: SNR-dependent high SME chi2 (bad fit) / FYI: median chi2_sp is 0.748
 256: Problems with Fe lines, where lineflux is not between 0.03 and 1.00
 512: sme did not finish
      a) no convergence == non-finite SPs
      b) Gaussian RV fit failed
      c) Timeout on ISAAC
1024: MARCS grid limit reached or outside of reasonable parameter range
Applying the following flags:
 1: Upper limit
 2: Bad chi2 fit
 4: Saturation
 8: Bad